# Méthodes de Krylov pour les éléments propres

Dans ce travail, on se propose d'implémenter méthode des sous-espaces de Krylov pour le calcul d'éléments propres. On va procéder par étapes en commençant par l'implémentation du procédé d'Arnoldi. **ATTENTION**: les matrices servant à faire les tests seront creuses, c'est-à-dire au format "sparse" de scipy. On prendra soin d'appeler les fonctions sous-jacentes aux variables pour faire des opérations comme le produit scalaire!

### Importation des librairies usuelles

In [ ]:
import numpy as np
import scipy as sp
import scipy.sparse as spsp
import numpy.linalg as npl
import scipy.sparse.linalg as spspl
import matplotlib.pylab as plt
%matplotlib inline

## Procédé d'Arnoldi

La première étape de ce travail consiste à implémenter le procédé d'Arnoldi. Soit $A\in M_n(\mathbb{R})$ et $b\in\mathbb{R}^n$. On rappelle que le procédé d'Arnoldi permet de construire des bases orthonormées des sous-espaces de Krylov

$$
\mathcal{K}_k(A,b) = \textrm{Vect}\{b,Ab,A^2b,\ldots,A^{k-1}b\}.
$$

Soit $(Q_k)_{k\geq 1}$ une suite de base orthonormée des sous-espaces $(\mathcal{K}_k(A,b))_{k\geq 1}$. On a de façon évidente

$$
Q_1 = [q^1] = [\,b/\|b\|_2\,]\in M_{n,1}(\mathbb{R}).
$$

On note $Q_k = [q^1,q^2,\ldots,q^k]\in M_{n,k}(\mathbb{R})$ où la famille $(q^j)_{1\leq j\leq k}$ est une base orthonormée de l'espace $\mathcal{K}_k(A,b)$. Pour passer de la matrice $Q_k$ à la matrice $Q_{k+1}$, on a la relation

$$
AQ_k = Q_{k+1} H_{(k+1,k)},
$$

où la matrice $H_{(k+1,k)}\in M_{k+1,k}(\mathbb{R})$ est de type Hessenberg. De manière itérative, étant donné la matrice $H_{(k,k-1)}$, le passage de $Q_k$ à $Q_{k+1}$ correspond à l'ajout d'un vecteur $q^{k+1}$ dans la base $Q_k$ mais aussi à l'agrandissement de la matrice $H$ par l'ajout d'une ligne de zéros de longueur $k$ puis d'une colonne de longueur $k+1$. Ainsi, lorsque l'on veut calculer le vecteur $q^{k+1}$ de la base $Q_{k+1}$, on fait

$$
p^{k+1} = Aq^k,
$$

puis l'on calcul les coefficient d'orthonogonalisation (qui correspondent aux $k$ premiers coefficients de la colonne à ajouter à la matrice $H$)

$$
h_{j,k} = \langle q^j,p^{k+1}\rangle,\quad \textrm{pour}\quad 1\leq j\leq k,
$$

et, finalement, on orthogonalise le vecteur $p^{k+1}$ par rapport à $Q_k$ pour déduire le vecteur $q^{k+1}$ comme suit

$$\begin{array}{ll}
r^{k+1} &= p^{k+1} - \sum_{j = 1}^k h_{j,k} q^j,
\\ h_{k+1,k} &= \|r^{k+1}\|_2
\\ q^{k+1} &= r^{k+1}/h_{k+1,k}.
\end{array}
$$

On propose d'implémenter une fonction `Step_Arnoldi` qui met en oeuvre l'ajout d'un vecteur $q^{k+1}$ à la base $Q_k$ et aussi l'ajout de la colonne $(h_{j,k})_{1\leq j\leq k+1}$ à la matrice $H_{(k,k-1)}$ pour $k\geq 2$. On remarque ainsi que l'on ne fera pas la première étape et, donc, que le vecteur $q^1$ (et donc $b$) sera déjà donné par l'utilisateur. Vous prendrez en arguments d'entrée:
- A: la matrice de la base de Krylov,
- Q: la matrice correspondant à la base $Q_k$,
- H: la matrice correspondant à la matrice $H_{(k,k-1)}$,

et en arguments de sortie:
- Q: la matrice correspondant à la base $Q_{k+1}$,
- H: la matrice correspondant à la matrice $H_{(k+1,k)}$.

Pour $k = 1$, la matrice $H_{(k,k-1)}$ n'est pas définie. On peut alors transmettre la variable d'entrée correspondante sous le format `None` et le traiter comme un cas à part dans la fonction.

> **À faire:** Implémenter la fonction `Step_Arnoldi` puis tester cette dernière en construisant une base de Krylov pour une matrice et un vecteur aléatoires de taille $10$. Vérifiez que la matrice de la base est bien orthogonale et que la relation matricielle $AQ_k = Q_{k+1} H_{(k+1,k)}$ est valide.

In [ ]:
def Step_Arnoldi(A,Q,H):
    n,k = np.shape(Q)
    h = np.zeros((k+1,1))
    q = np.zeros((n,1))
    p = A.dot(Q[:,-1])
    h[0:k,0] = Q.T.dot(p)
    p -= Q.dot(h[0:k,0])
    h[k,0] = npl.norm(p)
    if H is None:
        H = h
    else:
        m = np.zeros((1,k-1))
        H = np.concatenate((H,m),axis = 0)
        H = np.concatenate((H,h),axis = 1)
    p = p/h[k,0]
    q[0:n,0] = p
    Q = np.concatenate((Q,q),axis = 1)
    return Q,H

In [ ]:
N = 5
A = spsp.random(N,N,density=0.5)
b = np.random.random((N,1))
Q = np.zeros((N,1))
H = None
Q[:,0] = b[:,0]/npl.norm(b)
for j in range(N-1):
    Q,H = Step_Arnoldi(A,Q,H)
print(H)
print("Orthongonalité de Q:", np.allclose(Q.T.dot(Q),np.eye(N)))
print("Matrice de Hessenberg H:")
print(H)
print("Relation d'Arnoldi:", np.allclose(A.dot(Q[:,0:N-1]),Q.dot(H)))

## Méthode d'Arnoldi

On passe maintenant à la méthode d'Arnoldi. On rappelle que l'algorithme de cette méthode s'écrit formellement

$$
\left\{\begin{array}{ll}
b\in \mathbb{R}^n,\\
Q_0 = [\,b/\|b\|_2\,],\\
\textrm{Pour }k=1,2,\ldots\\\left\{\begin{array}{ll}
\textrm{Calculer}\; Q_{k+1}H_{(k+1,k)} = A Q_k\;\textrm{par Arnoldi},\\
\textrm{Calculer les valeurs propres de}\;H_{(k,k)},\\
\textrm{Test de convergence}.\\
\end{array}\right.
\end{array}\right.
$$

Le procédé d'Arnoldi permet de répondre directement à la première étape de la boucle sur $k$. Concernant la deuxième étape, on peut utiliser la méthode QR avec mise sous forme de Hessenberg, translation et déflation. On fournit ci-dessous des fonctions permettant de mettre en oeuvre le calcul des valeurs propres (et éventuellement vecteurs propres):

In [ ]:
def QR_HessenH(M):
    n = np.shape(M)[0]
    U = np.eye(n)
    R = np.copy(M)
    for k in range(n-1):
        x = R[k:k+2,k]
        l = npl.norm(x)
        c = x[0]/l
        s = x[1]/l
        for P in [U,R]:
            P_tmp_k = np.copy(P[k,:])
            P_tmp_kp1 = np.copy(P[k+1,:])
            P[k,:] = c*P_tmp_k + s*P_tmp_kp1
            P[k+1,:] = c*P_tmp_kp1 - s*P_tmp_k
    Q = U.T
    return Q,R

def Eig_QR_Defla(A, tol = 1e-7, itermax = None):
    n,m = np.shape(A)
    if n < 2:
        return A,0
    if itermax is None:
        itermax = 1000*n
    H = np.copy(A)
    mu = H[n-1,n-1]
    k = 0
    I = np.eye(n)
    Sd = np.zeros((n-1,1))
    L = []
    while (k<itermax):
        k += 1
        Q,R = QR_HessenH(H - mu*I)
        H = R.dot(Q) + mu*I
        H = np.triu(H,-1)
        mu = H[n-1,n-1]
        for l in range(n-1):
            Sd[l] = np.copy(H[l+1,l])
        i = np.argmin(abs(Sd))
        if abs(Sd[i])<tol:
            L1,ks1 = Eig_QR_Defla(H[0:i+1,0:i+1], tol, itermax)
            L2,ks2 = Eig_QR_Defla(H[i+1:n,i+1:n], tol, itermax)
            L = np.append(L,L2)
            L = np.append(L,L1)
            k = k + ks1 + ks2
            return(L,k)
    return np.diag(H),k

Il nous reste donc à aborder la question du test de convergence. Dans ce travail, on va uniquement se baser sur les valeurs propres obtenues et le but va être d'approximer un certain nombre $p$ de valeurs propres. Le problème est que les valeurs propres vont changer à chaque itération et ne pas être forcément ranger de la même manière. Ainsi, à l'itération $k$, on obtient les valeurs propres $(\lambda_j^{(k)})_{1\leq j\leq k}$ que l'on doit comparer avec les $(\lambda_j^{(k-1)})_{1\leq j\leq k-1}$ de l'itération précédente. On propose le critère de convergence suivant

$$
\textrm{Card}\left(\left\{\lambda_j^{(k)}; \frac{|\lambda_j^{(k)}-\lambda_i^{(k-1)}|}{|\lambda_i^{(k-1)}|}\leq \varepsilon\quad\textrm{pour un certain}\; i \right\}\right)\geq p,
$$

qui nous assurera que l'on a atteint la précision $\varepsilon$ pour au moins $p$ valeurs propres. En pratique, on peut former deux vecteur `a` et `b` puis utiliser la fonction `np.subtract.outer` pour obtenir l'ensemble des différences entre chaque coefficient de `a` et chaque coefficient de `b`. On obtient donc une matrice et il s'agit ensuite d'évaluer le nombre de coefficients inférieurs à $\varepsilon$ dans cette dernière.

> **À faire:** Implémenter la fonction `Arnoldi` qui met en oeuvre la méthode d'Arnoldi. Vous prendrez en arguments d'entrée:
- A: la matrice dont on cherche les valeurs propres,
- b: le vecteur pour la base de Krylov que l'on prendra de manière aléatoire si optionel,
- p: le nombre de valeurs propres que l'on désire approximées que l'on prendra égal à 1 si optionel,
- tol: la tolérance pour le critère d'arrêt que l'on prendra égal à 1e-3 si optionel.
En sortie, vous retournerez:
- L: une liste de vecteurs chacuns contenant les valeurs propres calculées à chaque itération,
- k: le nombre total d'itérations.

In [ ]:
def Arnoldi(A, b = None, p = 1, tol = 1e-3):
    n,m = np.shape(A)
    if b is None:
        b = np.random.random((n,1))
    k = 0
    b = np.reshape(b,(n,1))
    Q = np.zeros((n,1))
    Q[:,0] = b[:,0]/npl.norm(b)
    H = None
    L = []
    lt = 0
    for k in range(n):
        k += 1
        Q,H = Step_Arnoldi(A,Q,H)
        l,niter = Eig_QR_Defla(H[0:k,0:k])
        d = abs(np.subtract.outer(l,lt))
        if np.size(d[d<tol]) >= p:
            L += [l]
            return L,k
        lt = l
        L += [l]
    return L,k

> **À faire:** Tester la méthode d'Arnoldi sur une matrice creuse aléatoire de taille 200 et afficher l'évolution des valeurs propres en fonctions des itérations. Qu'observez vous?

In [ ]:
N = 200
D = spsp.diags(np.random.random(N))
E = 1e-3*spsp.random(N,N,density=0.25)
A = D+E
L,k = Arnoldi(A)
for i in range(np.size(L)):
    plt.scatter((i+1)*np.ones((i+1,1)),L[i])
plt.show()

## Méthode de Lanczos

Si la matrice $A$ est symétrique, le procédé d'Arnoldi se simplifie puisque l'on obtient une relation du type

$$
AQ_k = Q_{k+1} T_{(k+1,k)},
$$

où la matrice $T_{(k+1,k)}$ est tridiagonale. On note ses coefficients

$$
\alpha_k = T_{k,k}\quad\textrm{et}\quad \beta_{k+1} = T_{k+1,k} = T_{k,k+1}.
$$

En initialisant, $q^1 = b\,/\,\|b\|_2$, $q^0 = 0$ et $\beta_1 = 0$, le procédé d'Arnoldi devient alors le procédé de Lanczos que dont on écrit une étape, pour $k\geq 1$, comme

$$\begin{array}{ll}
v_k &= \; Aq^k - \beta_k q^{k-1},\\
\alpha_k & = \;\langle q^k,v_k \rangle,\\
w_k & = \;v_k - \alpha_k q^k,\\
\beta_{k+1} & =\; \|w_k\|_2,\\
q^{k+1} &= \; w_k / \beta_{k+1}.
\end{array}
$$

> **À faire:** Implémenter la fonction `Step_Lanczos` qui met en oeuvre le procédé de Lanczos. Vous prendrez en arguments d'entrée:
- A: la matrice dont on cherche les valeurs propres,
- Q: la matrice correspondant à la base $Q_k$,
- T: la matrice correspondant à la matrice $T_{(k,k-1)}$.

>En sortie, vous retournerez:
- Q: la matrice correspondant à la base $Q_{k+1}$,
- T: la matrice correspondant à la matrice $T_{(k+1,k)}$.

> Vous prendrez soin de traiter le cas $k = 1$ en initialisant convenablement le procédé. Vous pourrez d'ailleurs détecter ce cas en mettant la variable T d'entrée comme étant `None`.

In [ ]:
def Step_Lanczos(A,Q,T):
    if T is None:
        n,k = np.shape(Q)
        v = A.dot(Q[:,0])
        a = Q[:,0].dot(v)
        v -= a*Q[:,0]
        v = np.reshape(v,(n,1))
        b = npl.norm(v)
        Q = np.concatenate((Q,v/b),axis = 1)
        T = np.zeros((2,1))
        T[0,0] = a
        T[1,0] = b
    else:
        n,k = np.shape(Q)
        v = A.dot(Q[:,k-1]) - T[k-1,k-2]*Q[:,k-2]
        a = Q[:,k-1].dot(v)
        v -= a*Q[:,k-1]
        v = np.reshape(v,(n,1))
        b = npl.norm(v)
        Q = np.concatenate((Q,v/b),axis = 1)
        m = np.zeros((1,k-1))
        T = np.concatenate((T,m),axis = 0)
        l = np.zeros((k+1,1))
        l[k-2,0] = T[k-1,k-2]
        l[k-1,0] = a
        l[k,0] = b
        T = np.concatenate((T,l),axis = 1)
    return Q,T

> **À faire:** Tester la fonction `Step_Lanczos` en construisant une base de Krylov pour une matrice symétrique et un vecteur aléatoires de taille $10$. Vérifiez que la matrice de la base est bien orthogonale et que la relation matricielle $AQ_k = Q_{k+1} T_{(k+1,k)}$ est valide.

In [ ]:
N = 5
B = spsp.random(N,N,density=0.5)
A = 0.5*(B.T+B)
b = np.random.random((N,1))
Q = np.zeros((N,1))
T = None
Q[:,0] = b[:,0]/npl.norm(b)
for j in range(N-1):
    Q,T = Step_Lanczos(A,Q,T)
print("Orthongonalité de Q:", np.allclose(Q.T.dot(Q),np.eye(N)))
print("Matrice de tridiagonale T:")
print(T)
print("Relation de Lanczos:", np.allclose(A.dot(Q[:,0:N-1]),Q.dot(T)))

> **À faire:** Implémenter la fonction `Lanczos` qui met en oeuvre la méthode de Lanczos. Vous prendrez en arguments d'entrée:
- A: la matrice dont on cherche les valeurs propres,
- b: le vecteur pour la base de Krylov que l'on prendra de manière aléatoire si optionel,
- p: le nombre de valeurs propres que l'on désire approximées que l'on prendra égal à 1 si optionel,
- tol: la tolérance pour le critère d'arrêt que l'on prendra égal à 1e-3 si optionel.
En sortie, vous retournerez:
- L: une liste des vecteurs propres obtenus,
- k: le nombre total d'itérations.

In [ ]:
def Lanczos(A, b = None, p = 1, tol = 5e-1):
    n,m = np.shape(A)
    if b is None:
        b = np.random.random((n,1))
    k = 0
    b = np.reshape(b,(n,1))
    Q = np.zeros((n,1))
    Q[:,0] = b[:,0]/npl.norm(b)
    H = None
    Lt = 0
    for k in range(n):
        k += 1
        Q,H = Step_Lanczos(A,Q,H)
        L,niter = Eig_QR_Defla(H[0:k,0:k])
        d = abs(np.subtract.outer(L,Lt))
        if np.size(d[d<tol]) >= p:
            return L,k
        Lt = L
    return L,k

> **À faire:** Tester la méthode de Lanczos sur une matrice du laplacien en différences finies pour la recherche d'une seule valeur propre et comparer le résultat avec les valeurs exactes. Combien de valeurs propres sont approximées avec une erreur relative d'ordre $10^{-2}$? Et à l'ordre $10^{-3}$?

In [ ]:
def Laplacian(n) :
    return (n+1)**2*sp.sparse.diags([2*np.ones(n),-1*np.ones(n-1),-1*np.ones(n-1)], [0, -1, 1]).tocsc()
def Eig_Lapl(n):
    return np.array(4*((n+1)*np.sin(np.arange(1,n+1)*np.pi/(2*(n+1))))**2)

In [ ]:
n = 100
Lap = Laplacian(n)
L,niter = Lanczos(Lap)
L_exact = Eig_Lapl(n)
Err = abs(np.subtract.outer(L,L_exact))/L_exact
print("Nombre de valeurs propres approximées à l'ordre 1e-2:",np.size(Err[Err<1e-2]))
print("Nombre de valeurs propres approximées à l'ordre 1e-3:",np.size(Err[Err<1e-3]))